In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import statistics

In [ ]:
# Helper function to compute mean and CI
def mean_confidence_interval(data, confidence=0.95):
    data = list(data)
    n = len(data)
    if n < 2:
        # Not enough data points for CI
        mean_val = statistics.mean(data) if n > 0 else float('nan')
        return mean_val, mean_val, mean_val
    mean_val = statistics.mean(data)
    stdev = statistics.pstdev(data) if n == len(data) else statistics.stdev(data)
    # For 95% CI using normal approximation
    z = 1.96
    stderr = stdev / math.sqrt(n)
    lower = mean_val - z * stderr
    upper = mean_val + z * stderr
    return mean_val, lower, upper

# Parameters
csv_filename = 'simulation_results_eil51_6000_800_cooling_types_a280.csv'
cooling_types_of_interest = ["geometric", "linear", "logarithmic"]

# Read the CSV
df = pd.read_csv(csv_filename)

# Convert 'Distances', 'Temperatures', and 'Iterations' from string to list
df['Distances'] = df['Distances'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))
df['Temperatures'] = df['Temperatures'].str.strip("[]").str.split(", ").apply(lambda x: list(map(float, x)))
df['Iterations'] = df['Iterations'].str.strip("[]").str.split(", ").apply(lambda x: list(map(int, x)))

# Filter for the desired cooling types
df = df[df['Cooling Type'].isin(cooling_types_of_interest)]

# Group data by cooling type
cooling_types = df['Cooling Type'].unique()

aggregated = {}
for ct in cooling_types:
    subset = df[df['Cooling Type'] == ct]
    if len(subset) == 0:
        continue
    
    # Convert all runs distances and temperatures into DataFrames
    dist_df = pd.DataFrame(subset['Distances'].tolist())       # rows = runs, cols = iterations
    temp_df = pd.DataFrame(subset['Temperatures'].tolist())    # rows = runs, cols = iterations
    iter_df = pd.DataFrame(subset['Iterations'].tolist())      # rows = runs, cols = iterations

    n_runs = len(subset)

    # Compute mean and CI for distances and temperatures at each iteration
    dist_mean = dist_df.mean(axis=0)
    dist_std = dist_df.std(axis=0)
    dist_lower = dist_mean - 1.96 * dist_std / math.sqrt(n_runs)
    dist_upper = dist_mean + 1.96 * dist_std / math.sqrt(n_runs)

    temp_mean = temp_df.mean(axis=0)
    temp_std = temp_df.std(axis=0)
    temp_lower = temp_mean - 1.96 * temp_std / math.sqrt(n_runs)
    temp_upper = temp_mean + 1.96 * temp_std / math.sqrt(n_runs)

    # Compute average max iterations
    max_iterations_per_run = iter_df.apply(lambda x: x.iloc[-1], axis=1)
    avg_max_iterations = max_iterations_per_run.mean()

    aggregated[ct] = {
        'dist_mean': dist_mean,
        'dist_lower': dist_lower,
        'dist_upper': dist_upper,
        'temp_mean': temp_mean,
        'temp_lower': temp_lower,
        'temp_upper': temp_upper,
        'iterations': range(len(dist_mean)),
        'avg_max_iterations': avg_max_iterations,
        'n_runs': n_runs,
        'dist_df': dist_df
    }

# Plot Distance vs Temperature with CI
plt.figure(figsize=(12, 6))
for ct, data in aggregated.items():
    plt.plot(data['temp_mean'], data['dist_mean'], label=f'{ct} cooling')
    plt.fill_between(data['temp_mean'], data['dist_lower'], data['dist_upper'], alpha=0.2)
plt.title('Distance vs Temperature for Different Cooling Types (with 95% CI)')
plt.xlabel('Temperature (Decreasing)')
plt.ylabel('Distance')
plt.xscale('log')  
plt.gca().invert_xaxis()  # Reverse x-axis for decreasing temperature
plt.legend()
plt.grid(True)
plt.show()

# Plot Distance vs Iteration with CI
plt.figure(figsize=(12, 6))
for ct, data in aggregated.items():
    it = data['iterations']
    plt.plot(it, data['dist_mean'], label=f'{ct} Cooling')
    plt.fill_between(it, data['dist_lower'], data['dist_upper'], alpha=0.2)
plt.title('Distance over Iterations for Different Cooling Types (with 95% CI)')
plt.xlabel('Iterations')
plt.ylabel('Distance')
plt.legend()
plt.grid(True)
plt.show()

# Plot Stepwise Convergence (distance[i]-distance[i-1]) with CI
plt.figure(figsize=(12, 6))
for ct, data in aggregated.items():
    dist_runs = data['dist_df']
    dist_diff_runs = dist_runs.diff(axis=1).abs()  # Compute absolute difference across each run
    diff_mean = dist_diff_runs.mean(axis=0)
    diff_std = dist_diff_runs.std(axis=0)
    n_runs = data['n_runs']
    diff_lower = diff_mean - 1.96 * diff_std / math.sqrt(n_runs)
    diff_upper = diff_mean + 1.96 * diff_std / math.sqrt(n_runs)

    iterations = range(len(diff_mean))
    plt.plot(iterations, diff_mean, label=f'{ct} Cooling')
    plt.fill_between(iterations, diff_lower, diff_upper, alpha=0.2)

plt.title('Stepwise Convergence: Distance Changes Over Iterations (with 95% CI)')
plt.xlabel('Iterations')
plt.ylabel('Change in Distance (ΔDistance)')
plt.legend()
plt.grid(True)
plt.show()

# Zoom in on last 100 iterations for Distance vs Iteration
plt.figure(figsize=(12, 6))
for ct, data in aggregated.items():
    dist_mean = data['dist_mean']
    dist_lower = data['dist_lower']
    dist_upper = data['dist_upper']
    iterations = list(data['iterations'])

    start_idx = max(0, len(dist_mean)-100)

    plt.plot(iterations[start_idx:], dist_mean[start_idx:], label=f'{ct} Cooling')
    plt.fill_between(iterations[start_idx:], dist_lower[start_idx:], dist_upper[start_idx:], alpha=0.2)

plt.title('Distance vs Iterations (Last 100 Iterations) with 95% CI')
plt.xlabel('Iterations')
plt.ylabel('Distance')
plt.legend()
plt.grid(True)
plt.show()

# Zoom in on last 100 iterations for Stepwise Convergence
plt.figure(figsize=(12, 6))
for ct, data in aggregated.items():
    dist_runs = data['dist_df']
    dist_diff_runs = dist_runs.diff(axis=1).abs()
    diff_mean = dist_diff_runs.mean(axis=0)
    diff_std = dist_diff_runs.std(axis=0)
    n_runs = data['n_runs']
    diff_lower = diff_mean - 1.96 * diff_std / math.sqrt(n_runs)
    diff_upper = diff_mean + 1.96 * diff_std / math.sqrt(n_runs)

    start_idx = max(0, len(diff_mean)-100)

    iterations = range(len(diff_mean))
    plt.plot(iterations[start_idx:], diff_mean[start_idx:], label=f'{ct} Cooling')
    plt.fill_between(iterations[start_idx:], diff_lower[start_idx:], diff_upper[start_idx:], alpha=0.2)

plt.title('Stepwise Convergence (Last 100 Iterations) with 95% CI')
plt.xlabel('Iterations')
plt.ylabel('Change in Distance (ΔDistance)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Zoom in on the last 5 degrees of temperature: Separate subplots for each cooling type
num_ct = len(aggregated)
fig, axes = plt.subplots(nrows=1, ncols=num_ct, figsize=(5 * num_ct, 6))

if num_ct == 1:
    # If there's only one cooling type, just use axes directly
    axes = [axes]

for ax, (ct, data) in zip(axes, aggregated.items()):
    temp_mean = data['temp_mean']
    dist_mean = data['dist_mean']
    dist_lower = data['dist_lower']
    dist_upper = data['dist_upper']

    # Identify final temperature (lowest temperature)
    final_temp = temp_mean.iloc[-1]
    # Define cutoff as final_temp + 5
    cutoff_temp = final_temp + 5

    # Get indices where temperature <= cutoff_temp
    indices = [i for i, t in enumerate(temp_mean) if t <= cutoff_temp]

    if len(indices) > 0:
        start_idx = indices[0]
        # Plot only this section
        ax.plot(temp_mean[start_idx:], dist_mean[start_idx:], label=f'{ct} Cooling')
        ax.fill_between(temp_mean[start_idx:], dist_lower[start_idx:], dist_upper[start_idx:], alpha=0.2)

    ax.set_title(f'{ct} Cooling (Last 5° Above Final Temp)')
    ax.set_xlabel('Temperature')
    ax.set_ylabel('Distance')
    ax.set_xscale('log')
    ax.invert_xaxis()
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Zoom in on last 100 iterations: Separate subplots for each cooling type
num_ct = len(aggregated)
fig, axes = plt.subplots(nrows=1, ncols=num_ct, figsize=(5 * num_ct, 6))

if num_ct == 1:
    # If there's only one cooling type, wrap it in a list
    axes = [axes]

for ax, (ct, data) in zip(axes, aggregated.items()):
    dist_mean = data['dist_mean']
    dist_lower = data['dist_lower']
    dist_upper = data['dist_upper']
    iterations = list(data['iterations'])

    # Determine the starting index for the last 100 iterations
    start_idx = max(0, len(dist_mean)-100)

    # Plot the last 100 iterations
    ax.plot(iterations[start_idx:], dist_mean[start_idx:], label=f'{ct} Cooling')
    ax.fill_between(iterations[start_idx:], dist_lower[start_idx:], dist_upper[start_idx:], alpha=0.2)

    ax.set_title(f'{ct} Cooling (Last 100 Iterations)')
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Distance')
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.show()
